# 一、项目概述

## 1.1 项目背景

哔哩哔哩（bilibili）是中国知名的ACG内容平台及线上宅文化社区，其动漫板块以季度更新的"番剧"为主要形式。  
本项目通过分析B站动漫数据，探究以下方面：

- **时间空间的对比分析**：不同地区/年份的番剧数量变化
- **番剧质量与口碑评价**：评分分布与追番人数关系
- **热门番剧的类型演变**：题材偏好随时间的变化趋势

### 数据特征
- **追番人数**：反映作品热度与观众数量
- **作品评分**：0-10分制，是质量的评价指标
- **番剧类型**：不同番剧的题材和内容(日常/热血/漫改等)

数据来源：B站动漫番剧排行榜(以追番人数从高到低为索引):  
<https://www.bilibili.com/anime/index/?from_spmid=666.4.index.0#st=1&order=3&season_version=-1&spoken_language_type=-1&area=-1&is_finish=-1&copyright=-1&season_status=-1&season_month=-1&year=-1&style_id=-1&sort=0&page=1>

## 1.2 可视化内容

1. **地区数量分析**
   - 各地区每年上映番剧数量分布（对数刻度条形图）、占比分布（扇形图）

2. **评分趋势分析**
   - 评分密度分布与直方图、年度平均评分趋势（折线、条形复合图）

3. **观众行为分析**
   - 年度番剧数量vs追番人数趋势（双折线图）、评分与追番人数关系（气泡散点图）

4. **类型变化分析**
   - 所有番剧vs高分番剧(≥9.5)类型词云对比、1980-2020期间三个时间段的类型词云对比

## 1.3 技术实现

### 功能模块
| 模块        | 描述                        |
|-------------|---------------------------- |
| 数据收集    | 使用公开合法接口、构建爬虫    |
| 数据清洗    | 缺失值处理与异常值修正等      |
| 数据分析    | 多因素统计分析与可视化呈现    |

### 技术支持

|   分析模块     |   所需库名   | 作用描述                                                                 |
|---------------|--------------|--------------------------------------------------------------------------|
| **数据采集**  | `requests`   | 封装 HTTP 请求，支持会话管理、认证、代理等功能，适配 B 站反爬机制。            |
|               | `BeautifulSoup` | 解析 HTML 文档，精准定位番剧信息（标题、评分、播放量等），处理动态加载内容。 |
| **数据处理**  | `pandas`     | 结构化数据处理核心工具，用于数据清洗、特征工程及初步分析。                |
|               | `numpy`      | 提供多维数组支持，加速数值计算（如评分统计、时间序列分析）。             |
| **可视化**    | `matplotlib` | 基础绘图库，用于绘制条形图、折线图等，支持自定义配色与样式。            |
|               | `seaborn`    | 增强统计图表美观性，简化复杂图表（如多变量关系图、热力图）的绘制流程。    |
|               | `wordcloud`  | 生成词云图，直观展示番剧类型高频词汇及主题演化。                       |
|               | `missingno`  | 可视化缺失值分布，辅助判断数据完整性（如评分字段的缺失比例）。          |
| **辅助工具**   | `re`         | 正则表达式处理文本数据（如提取番剧副标题、制作地区标签映射）。          |
|               | `json`       | 序列化中间数据，支持与 B 站 API 交互及结果持久化存储。                    |


# 二、 数据预处理

## 2.1 数据获取过程说明

本项主要爬取 **B站动漫番剧排行榜(以追番人数从高到低为索引)**:  
<https://www.bilibili.com/anime/index/?from_spmid=666.4.index.0#st=1&order=3&season_version=-1&spoken_language_type=-1&area=-1&is_finish=-1&copyright=-1&season_status=-1&season_month=-1&year=-1&style_id=-1&sort=0&page=1>

> ![Blibli页面概览](blibli_page_overview1.png)


### 初始爬虫方案
尝试通过 Python 爬虫获取数据，具体流程有：
1. **列表页爬取**：获取动漫排行榜的大致信息及详情页直达链接
2. **详情页抓取**：访问每个动漫的独立页面获取完整数据
3. **数据提取**：
   - 使用 `requests` 库获取页面 HTML 中的 JSON 数据
   - 通过 `Beautiful Soup` 解析并提取关键字段

### 最终数据来源
由于实际爬取过程中遇到大规模详情页请求的频率限制,最终采用B站动漫结构化数据集（来源：GitHub 公开项目）作为数据源，使用公开、合法的数据集有以下优势：  
1.数据结构规范（与官方API一致）  
2.经过社区验证的数据准确性高  

完整包含 2811 条 B 站官方排行榜数据，部分数据格式为:

| 序号 | title               | subTitle           | score | order        | time                | table                 | area |
| ---- | ------------------- | ------------------ | ----- | ------------ | ------------------- | --------------------- | ---- |
| 0    | 鬼灭之刃 刀匠村篇   | 信念锻刃，尽斩恶鬼 | 9.2   | 1564.5万追番 | 2023-06-03 20:00:00 | 漫画改 奇幻 热血 战斗 | 日本 |
| 1    | 鬼灭之刃 游郭篇     | 斩断鬼所栖身的夜   | 9.7   | 1409.3万追番 | 2022-03-05 20:00:00 | 漫画改 热血 战斗 奇幻 | 日本 |
| 2    | 鬼灭之刃 无限列车篇 | 潜入无限的梦境中   | 9.8   | 1333万追番   | 2021-11-17 20:00:00 | 漫画改 热血 战斗 奇幻 | 日本 |
| 3    | 鬼灭之刃            | 谁都斩不断的羁绊   | 9.7   | 1267.4万追番 | 2019-04-07 00:00:00 | 漫画改 战斗 热血 声控 | 日本 |
| 4    | 咒术回战 第二季     | 天上天下，唯我独尊 | 9.4   | 1242.3万追番 | 2023-08-24 20:00:00 | 漫画改 热血 战斗 奇幻 | 日本 |


## 2.2 数据清洗

#### 用图形可视化方式展示 df 中缺失值（NaN）的位置,快速观察缺失数据的分布、模式等特性.
```Python
msno.matrix(df, labels=True)
```

![Graphical_visualization1](Graphical_visualization1.png)

#### 统计每一列中有多少个缺失值（NaN）,便于进一步填补或删除.
```Python
df.isna().sum()
```
| Column    | Missing Values |
|-----------|----------------|
| title     | 0              |
| subTitle  | 899            |
| score     | 136            |
| order     | 0              |
| time      | 0              |
| table     | 55             |
| area      | 0              |


### 缺失值处理

#### 1. subTitle列（副标题）
```python
df.drop(columns='subTitle', inplace=True)  
```
- **空缺量**：899条
- **决策依据**：
  - substitute副标题对分析数据作用不大
  - subTitle为空的动漫占比约三分之一,占比影响较大;

#### 2. score列（评分）
```python
df.dropna(subset=['score'], inplace=True)  
```
- **空缺量**：136条
- **空缺原因**：
  1. 目前还未上架
  2. 作品冷门,评分不足
- **决策依据**：数量符合常理,可保留score列，只删除该列有NaN的行;

#### 3. table列（类型）
- **空缺量**：55条
- **处理方案**：暂时保留，后续进行处理
- **决策依据**：table为空的数据不多,且直接删去会影响较多其他列数据

### 数据格式处理

#### 1.order列（排名）
* 去掉order列里的'万追番','万追剧';检查是否存在'万'字,因为排行榜追剧/追番人数必然以万字计
```Python
df['order'] = df['order'].str.replace('万追番', '')
df['order'] = df['order'].str.replace('万追剧', '')
has_wan = df['order'].str.contains('万 ', na=False)
print('包含“万”的行数:', has_wan.sum())
print(df[has_wan])
```

#### 2. time列（时间）
* time包含年,日,月,时,分,秒以及空格和其他符号,将其表示为只含年份的纯数字 
```Python
df['time'] = df['time'].str[:4]
```

#### 将清洗好的数据放入excel文件中,再次统计每一列中有多少个缺失值（NaN）
```Python
df.to_excel('动漫数据_已处理2.xlsx')
df.isna().sum()
```
| Column    | Missing Values |
|-----------|----------------|
| title     | 0              |
| subTitle  | 0              |
| score     | 0              |
| order     | 0              |
| time      | 0              |
| table     | 0              |
| area      | 0              |

#### table为NaN数小于55,说明在清除score为NaN的行时或清除重复的title时,同时清空了table为NaN的行  

#### 数据清洗完成,接下来开始数据分析

# 三、数据分析

## 3.1 动漫番剧地区分布条形图

```Python
plt.figure(figsize=(8,5))
area_counts=area_value_counts.reset_index()
area_counts.columns=['area','count']
sns.barplot(area_counts, x='area', y='count', palette='viridis')
plt.yscale('log')  
plt.title('各地区动漫番剧数量分布（对数刻度）')
plt.xlabel('地区')
plt.ylabel('动漫番剧数量（对数）')
plt.show()
```
![Number_of_logs2](Number_of_logs2.png)

## 3.2 动漫番剧地区分布扇形图
```Python
plt.figure(figsize=(6,6))
most_area=area_counts[area_counts['count']>25]
other=pd.DataFrame({
    'area':['其他'],
    'count':[area_counts[area_counts['count']<=25]['count'].sum()]
})
plot_data=pd.concat([most_area,other])
plt.pie(
    plot_data["count"],
    labels=plot_data['area'],
    autopct='%.1f%%',
    colors=plt.cm.Pastel1.colors,
    pctdistance=0.9,
    startangle=140
        )
plt.title('B站动漫番剧地区分布(数量≤25的归为\'其他\')')
plt.show()
```
![Regional_distribution3](Regional_distribution3.png)

## 主要国家占比（扇形图）:
日本 95.4%,美国 3.1%,英国 1%, 其他 0.4%

**原因**：
1. 日本动画产业成熟，产业链完整
2. 美国主攻3D动画电影
3. 欧洲以艺术短片为主
4. 其他国家产量较少

### 对数柱状图扩展

1. **数量差距太大**  :日本产量是其他国家的几百倍,普通柱状图会让小国家完全看不见

2. **看清所有国家**  :对数缩放后，小国家的数据也能显示,方便比较不同规模的市场

3. **发现潜力市场**  :能看到虽然占比小但增长快的国家,比如中国、韩国等新兴市场

### **总结**：扇形图反映整体，对数柱状图反映细节，两者结合可以全面了解行业分布。

## 3.3 动漫番剧评分密度和直方分布图

```Python
plt.figure(figsize=(8,5))
sns.histplot(df['score'],bins=20,kde=True,color='steelblue')
plt.title('动漫番剧评分的密度分布和直方图')
plt.xlabel('评分')
plt.ylabel('番剧数量')
plt.show
```
![Density_histogram4](Density_histogram4.png)

## 评分分布分析
整体评分集中在 **9.5 分** 附近，部分作品甚至能达到 **9.9 分**。

### 评分偏高的原因
1. **评分门槛机制**  
  B 站要求用户必须看完该集才能评分，自然筛选掉了中途弃番的用户，导致评分样本偏向于"能看下去"的观众。
2. **粉丝滤镜效应**  
  动漫类作品的评分不仅反映制作质量，还会受到原作情怀、IP 影响力等外部因素影响。

### 评分参考标准(B站用户约定俗成的特色指标)
| 评分区间 | 作品评价                     |
|----------|----------------------------|
| 9.9      | 神作                     |
| ≥9.5     | 整体优秀，有微小瑕疵        |
| ≥9.0     | 质量过关，无需深究逻辑      |
| <8.0     | 存在明显制作问题         |

## 3.4 动漫番剧数量和评分趋势图

```Python
grouped=df.groupby('time').agg({'title':'count','score':'mean'}).reset_index()
grouped = grouped[(grouped['time'] >= 1980) & (grouped['time'] <= 2020)]
fig,ax1=plt.subplots(figsize=(10,6))
ax2=ax1.twinx()
ax1.bar(grouped['time'],grouped['title'],color='skyblue',label='番剧数量')
ax2.plot(grouped['time'],grouped['score'],color='orange',label='平均评分',marker='o')
ax1.set_xlabel('年份')
ax1.set_ylabel('番剧数量')
ax2.set_ylabel('平均评分')
ax1.legend()
ax2.legend()
plt.title('每年动漫番剧数量与平均评分趋势')
plt.show()
```
![Quantity_scoring5](Quantity_scoring5.png)

```Python
grouped=df.groupby('time').agg({'title':'count','order':'sum'}).reset_index()
grouped = grouped[(grouped['time'] >= 1980) & (grouped['time'] <= 2020)]
fig,ax1=plt.subplots(figsize=(10,6))
ax2=ax1.twinx()
ax1.plot(grouped['time'],grouped['title'],color='skyblue',label='番剧数量',marker='o')
ax2.plot(grouped['time'],grouped['order'],color='coral',label='追番人数',marker='o')
ax1.set_xlabel('年份')
ax1.set_ylabel('番剧数量')
ax2.set_ylabel('追番人数')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.title('每年动漫番剧数量与追番人数趋势')
plt.show()
```
![Number6](Number6.png)

```Python
plt.figure(figsize=(8,5))
sns.scatterplot(data=df,x='order',y='score',alpha=0.8,palette='Set2',hue='area',size='order',sizes=(20,200))
plt.title='评分与追番人数关系(气泡大小=追番人数)'
plt.xlabel('追番人数(万)')
plt.ylabel=('评分')
plt.legend(bbox_to_anchor=(1.05,1),loc='upper left')
plt.tight_layout()
plt.show()
```
![scatter7](scatter7.png)

## 动漫番剧与观众趋势分析（1980-2020）

### 产量与观众规模
- **整体发展**：
  1. 动画产业呈现持续扩张态势,2000年后进入加速发展阶段
  2. 观众规模的增长存在明显滞后性,在产业扩张约十年后才出现显著提升,在2020年达到顶峰.
  3. 2010年是产业规模与受众群体同步爆发的关键时期

- **三个发展阶段**：
  - **萌芽期（1980-1999）**：产业规模有限，年产量维持在较低水平
  - **扩张期（2000-2009）**：制作数量开始稳定增长
  - **爆发期（2010-2020）**：产量突破历史峰值，观众规模呈指数级增长
  

### 产量与作品质量关系
- **长期趋势**：
  1. 产量增长与平均评分存在明显负相关
  2. 早期（1980年代）作品普遍获得较高评价
  3. 2010年后随着产量激增,平均评分波动加剧且整体下降

- **关键点**：
  1. 2013年前后：产量突破100部,平均评分开始持续下降
  2. 2019年：平均评分达到历史低位,市场出现饱和迹象

### 受众规模与作品质量
- **整体规律**：
  1. 大众关注度与作品质量呈现正相关关系
  2. 高人气作品（追番人数>1000万）均保持较高评分基准
  3. 中低关注度区间存在明显的质量分化现象:  
  - 大部分番剧的追番人数在0-400万,导致该区域气泡密集,同时该区域内小部分番剧评分低于8分;  
  - 追番人数400-1000万的番剧大部分评分在9分以上
- **地域分布**：
  1. 日本作品占据绝对主导地位
  2. 非日本产区的作品普遍关注度较低,但质量分布区间较大

## 3.5 动漫番剧类型词云对比

```Python
import re
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt
font_path='./simhei.ttf' 
def clean_type(text):
    """清洗类型内部文本，保留中英文并去除特殊符号"""
    if pd.isna(text):
        return ''
    return re.sub(r'[^\u4e00-\u9fa5a-zA-Z]', '', str(text)).strip()

def get_type_counts(df):
    types = df['table'].dropna().str.split(' ', expand=True).stack()
    cleaned = types.apply(clean_type)
    return cleaned[~cleaned.isin(['未知', '其他']) & (cleaned != '')].value_counts()

df['score'] = pd.to_numeric(df['score'], errors='coerce')

# 全局词云和高分词云
all_freq = get_type_counts(df)
high_freq = get_type_counts(df[df['score'] >= 9.5])

# 第一张图：所有番剧和评分≥9.5的词云
fig1, axes1 = plt.subplots(1, 2, figsize=(10, 4))
for ax, freq, title in zip(axes1, [all_freq, high_freq], ['所有番剧类型', '评分≥9.5类型']):
    wc = WordCloud(width=800, height=400, background_color='black', font_path=font_path).generate_from_frequencies(freq.to_dict())
    ax.imshow(wc)
    ax.set_title(f'{title}词云', fontsize=16)
    ax.axis('off')

plt.tight_layout()
plt.show()

time_ranges = [
    ('1980-1995', (1980, 1995)),
    ('1995-2010', (1995, 2010)),
    ('2010-2020', (2010, 2020))
]

# 第二张图：三个时间段的词云
fig2, axes2 = plt.subplots(1, 3, figsize=(18, 5))
for ax, (time_range, (start, end)) in zip(axes2, time_ranges):
    time_df = df[(df['time'] >= start) & (df['time'] <= end)]
    time_freq = get_type_counts(time_df)
    
    wc = WordCloud(width=800, height=400, background_color='black', 
                  font_path=font_path).generate_from_frequencies(time_freq.to_dict())
    ax.imshow(wc)
    ax.set_title(f'{time_range}年份番剧类型词云', fontsize=16)
    ax.axis('off')

plt.tight_layout()
plt.show()

print("=== 各分组类型出现次数统计 ===")

```
![wordcloud8](wordcloud8.png)
![wordcloud10](wordcloud10.png)

## 动漫番剧类型词云图分析

### 所有番剧与高分番剧对比

**主流类型**:
1. 主流类型奇幻、日常和热血类作品在高分番剧中仍保持稳定优势。

2. 日常类作品在高分段占比提升最为显著，漫画改编作品的高分出现率也明显优于平均水平。

**质量特征**:
1. 情感向作品整体表现优异，其中治愈、萌系、搞笑类作品更容易获得高评价。

2. 架空、战斗等制作要求高的类型呈现精品化趋势。

### 不同时期类型演变

- **1980-1995黄金期特征**:
科幻和热血类作品占据主导地位，传统少年向题材盛行，整体类型选择较为有限。

- **1995-2010过渡期变化**:
日常系、治愈类作品开始崛起，热血、奇幻、科幻等传统类型仍占据主流，类型细分趋势加剧。

- **2010-2020新时期发展**:
日常类成为最主流类型，漫画改编作品快速增长，少女、少儿类型逐渐减少,类型融合成为创作新趋势。

### 多维演化动因

1. **市场需求**：观众偏好从热血型向日常型转变，反映社会压力增大背景下对轻松内容的需求增长

2. **技术支持**：制作技术进步使得高精度类型能够实现精品化，推动行业标准提升

3. **内容变化**：漫画、小说改编等新形式的崛起改变了类型分布，带来新的创作视角



# 四、结论与建议

## 调查的主要发现

1. **市场格局高度集中**  
   - 日本动漫占据B站番剧总量的95.4%，形成垄断地位  
   - 美国(3.1%)和英国(1%)等非日本产区作品数量有限，但质量稳定  
   - 国产动漫在数量上仍有较大发展空间  

2. **质量与热度呈现正向关联**  
   - 高分作品(≥9.5)具有更高的追番人数  
   - "马太效应"：优质作品获得超量关注  
   - 中等热度区间(400-1000万追番)作品质量分化明显  

3. **产业演变呈现阶段性特征**  
   - **1980-2010年**：产量缓慢增长，平均评分保持高位  
   - **2010-2020年**：产量爆发式增长，平均评分波动下降  
   - 观众数量增长慢于产业扩张约5-8年  

4. **类型偏好发生显著改变**  
   - 从传统"热血/奇幻"主导转向，到"日常/治愈"迅速增长  
   - 漫画改编作品在高分区间的占比有极大提升  
   - 2010年后，多种类型融合成为新趋势  

## 动漫行业启示

1. **对平台的建议**  
   - 建立更精确、公平的评分权重机制，减少粉丝效应对评价的影响  
   - 加强非日本产区优质作品的引入和推广   

2. **对制作方的建议**  
   - 推进类型融合创新，避免同质化
   - 平衡产量与质量的关系，减少过度商业化的影响  
   - 重视漫画/小说改编等新形式作品的开发价值    
   - 评估作品时应结合平台用户特点（B站的评分相较于其他平台普遍偏高）  

## 调查的局限与展望

1. **数据采集**  
   - 仅包含B站官方排行榜数据和看完用户的评价  

2. **研究方法**   
   - 未考虑番剧时长、制作成本等潜在影响因素  

3. **未来展望**  
   - 引入多平台数据对比分析  
   - 建立跨年度、跨地区的综合评价体系  

#### 本项目表明，B站动漫已形成独特生态，其类型演变反映了当代中国年轻观众的审美变迁。
#### 未来随着国产动漫的崛起和各大媒体竞争加剧，这一格局可能面临挑战与重构，值得持续跟踪调查。
